# One step further, for a massive database of songs 
# WARNING: Big loading times zone 

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from getpass import getpass
from tqdm.notebook import tqdm
import pandas as pd
import random 

In [2]:
c_id = str(getpass())

········


In [3]:
c_s = str(getpass())

········


In [4]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=c_id,
                                                          client_secret=c_s))

We test that is running properly. We will start taking a list of categories. We test if the calls work

And we put our categories together

In [5]:
categories = []
for cat in sp.categories(limit = 50)['categories']['items']:
    categories.append(cat['id'])

In [6]:
len(categories)

39

In [7]:
playlist_id = []
for cat in tqdm(categories):
    try:
        i=0
        cat_data = sp.category_playlists(cat, limit = 50, offset=i)['playlists']['items']
        #Having this while loop will allow retrieve more than 50 per category 
        while len(cat_data) == i+50:
            i +=50
            cat_data += sp.category_playlists(cat, limit = 50, offset=i)['playlists']['items']   
    except:
        continue
    for plist in cat_data:
        try:
            playlist_id.append(plist['id'])
        except:
            continue

  0%|          | 0/39 [00:00<?, ?it/s]

In [8]:
len(playlist_id)

793

In [9]:
# checkpoint1:saving playlist_id
out_playlist_id = pd.Series(playlist_id, name='playlist_id')
out_playlist_id.to_csv("../data/checkpoints/checkpoint_playlist_id.csv")

Here we will take all artist ids

In [10]:
artist_ids = []
for plist in tqdm(playlist_id):
    i = 0
    tracks = sp.playlist_tracks(plist, limit=100, offset=i)['items']
    while len(tracks)==i+100:
        i += 100
        tracks += sp.playlist_tracks(plist, limit=100, offset=i)['items']
    for track in tracks:
        try:
            for ids in [artist['id'] for artist in track['track']['artists']]:
                artist_ids.append(ids)
        except:
            continue

  0%|          | 0/793 [00:00<?, ?it/s]

Removing duplicated ids

In [11]:
artist_ids = list(set(artist_ids))
print(len(artist_ids))

28227


In [12]:
# checkpoint2:saving artist_ids
out_artist_ids = pd.Series(artist_ids, name='artist_ids')
out_artist_ids.to_csv("../data/checkpoints/checkpoint_artist_ids.csv")

And shuffle the list to reduce bias from the sample recovered. If it is not shuffled, the playlists from certain category would be taken first and the categories towards the end, as this script takes a lot of time to complete, won't be considered. This way we can run the script to gather as much data as we can in the period of time, and will be tracks from random artists instead of the ones from certain category. 

In [13]:
random.shuffle(artist_ids)

Now we get a list of all their albums

In [14]:
album_ids = []

for artist in tqdm(artist_ids):
        i = 0
        albums = sp.artist_albums(artist, limit=50, offset=i)['items']
        while len(albums)==i+50:
            i += 50
            albums += sp.artist_albums(artist, limit=50, offset=i)['items']
        for album in albums:
            album_ids.append(album['id'])

    

  0%|          | 0/28227 [00:00<?, ?it/s]

Again, removing duplicates

In [15]:
len(album_ids)

2687991

In [16]:
album_ids = list(set(album_ids))
len(album_ids)

1110365

In [17]:
# checkpoint2:saving artist_ids
out_album_ids = pd.Series(album_ids, name='album_ids')
out_album_ids.to_csv("../data/checkpoints/checkpoint_album_ids.csv")

In [18]:
random.shuffle(album_ids)

Going through every album and extracting the track information

In [19]:
song_id = []
song_name = []
song_artists = []



for album in tqdm(album_ids):
    try:
        tracks = sp.album_tracks(album)['items']            
    except:
        continue
    for track in tracks:
        song_id.append(track['id'])
        song_name.append(track['name'])
        song_artists.append([artist['name'] for artist in track['artists']])

  0%|          | 0/1110365 [00:00<?, ?it/s]

Creating a pandas dataframe, and dropping duplicates, if the same track was in several albums

In [20]:
data = {'id':song_id,
        'name':song_name,
        'artists':song_artists
        }
songs = pd.DataFrame(data=data)
songs

,id,name,artists
0,5B38hxdKziEKVO0HO31wg7,"Donizetti: Francesca di Foix: ""Senti senti ......","[Gaetano Donizetti, Antonello Allemandi, Londo..."
1,2ZeKTqsCuQhqJpjFFbJvl9,"Donizetti: Francesca di Foix: ""Quest'e il loco...","[Gaetano Donizetti, Antonello Allemandi, Londo..."
2,365ZW44z5LTdZ8RUC6b7eM,"Donizetti: Francesca di Foix: ""Ecco il Conte ....","[Gaetano Donizetti, Antonello Allemandi, Londo..."
3,3I1PR5GJ004cf0z7acH8SO,"Donizetti: Francesca di Foix: ""Grato accolse i...","[Gaetano Donizetti, Antonello Allemandi, Londo..."
4,1QZhXEa9R4Y4IesZQhbdIL,"Donizetti: Francesca di Foix: ""Oh quale apport...","[Gaetano Donizetti, Antonello Allemandi, Londo..."
...,...,...,...
11870010,11ZsxFaDHZDlhSeZO8KMgo,M.O.B.,[BlocBoy JB]
11870011,3aayNvTj79fO5HrWJtNAXx,T'd Off,[BlocBoy JB]
11870012,0FqlIQoWNeIQTy5flsd40b,Last Day Out,[BlocBoy JB]
11870013,6z9LOvbHFza6sWtPz2d8oZ,Lilium,[DJ Seinfeld]


In [21]:
songs = songs.drop_duplicates(subset='id')
songs.reset_index(drop=True, inplace=True)

In [22]:
# checkpoint:saving songs
songs.to_csv("../data/checkpoints/songs.csv")

In [23]:
def chunks(lst,n):
    """
    Yields n-sized chunks of lst
    """
    for i in range(0, len(lst), n):
        yield lst[i:i+n]



In [25]:
song_ids = songs['id']

In [ ]:

audio_feats = []
for chunk in tqdm(list(chunks(song_ids,50))):
    try:
        audio_feats.append(sp.audio_features(chunk))
    except:
        continue

  0%|          | 0/237401 [00:00<?, ?it/s]

we flatten this list into one long list of dictionaries, and turn it into a series, in order to use pandas.dropna(), so the conversion to data frame can easily work 

In [ ]:
audio_feats2 = pd.Series([y for x in audio_feats for y in x], name='features')

In [ ]:
feat_df = pd.DataFrame(list(audio_feats2.dropna()))

In [ ]:
# checkpoint:saving features
feat_df.to_csv("../data/checkpoints/feat_df.csv")
audio_feats2.to_csv("../data/checkpoints/audio_feats2.csv")

In [ ]:
final = songs.merge(feat_df, on = 'id')

We test that the merge worked properly using a random sample of 50, taking the audio features again, and comparing the value of danceability, for example in the resulting dataframe. The next cell will return true if there are matches, false if not 

In [ ]:
sample = final.sample(50).reset_index()
(pd.DataFrame(sp.audio_features(sample['id']))['danceability'] == sample['danceability']).value_counts()

In [ ]:
final.to_csv("../data/checkpoints/Final dataframe.csv")